In [3]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [4]:
## Hyperparameters
batch_size = 4
n = 5
din = 8
dout = 4
n_heads = 2

In [ ]:
class multihead(nn.Module):
    def __init__(self,din,dout,n_heads,device):
        super(multihead,self).__init__()
        # standard stuff
        self.din = din
        self.dout = dout
        self.n_heads = n_heads
        self.dheads = int(dout/n_heads)
        self.device = device
        # initializing with nn.Linear for more robus intitializations
        self.query = nn.Linear(din,dout,device = device)
        self.key = nn.Linear(din,dout,device = device)
        self.value = nn.Linear(din,dout,device = device)
        self.softmax = nn.Softmax(dim  = -1)
    def forward(self,X):
        batch_size,n,din = X.shape
        # getting the q,k,v matrices
        q = self.query(X)
        k = self.key(X)
        v = self.value(X)
        # splitting accross heads
        q = q.view(batch_size,n,self.n_heads, self.dheads)
        k = k.view(batch_size,n,self.n_heads, self.dheads)
        v = v.view(batch_size,n,self.n_heads, self.dheads)
        # viewing head wise
        q = torch.transpose(q,1,2)
        k = torch.transpose(k,1,2)
        v = torch.transpose(v,1,2)
        # standard q.kT
        attn_scores = q @ k.transpose(2,3)
        mask =  torch.triu(torch.ones(n,n),diagonal=1).bool()
        #causal attention
        attn_scores = torch.masked_fill(attn_scores/(self.dheads**0.5),mask, -torch.inf)
        attn_weights = self.softmax(attn_scores)
        # standard qk.T @ v
        context_vector = attn_weights @ v
        context_vector = context_vector.transpose(1,2)
        # now viewing token wise
        context_vector = context_vector.reshape(batch_size,n,self.dout)
        return context_vector

In [66]:
device = "cpu"

In [67]:
embeddings = torch.randn(batch_size,n,din)

In [68]:
attention = multihead(din,dout,n_heads,device)

In [69]:
context_vector = attention(embeddings)

In [70]:
context_vector.shape

torch.Size([4, 5, 4])